# KISS Agent Framework - Interactive Tutorial

**Version:** 0.1.7

This notebook provides an interactive tutorial for the KISS (Keep It Simple, Stupid) Agent Framework.

KISS is a lightweight agent framework that implements a ReAct (Reasoning and Acting) loop for LLM agents.

## Prerequisites

Before running this notebook, ensure you have:
1. Installed the KISS framework: `uv sync --group dev`
2. Set up your API keys as environment variables

**Note:** Some cells require API keys to run (GEMINI_API_KEY, OPENAI_API_KEY, ANTHROPIC_API_KEY, etc.)

## 1. Setup and Environment Check

First, let's check that the KISS framework is properly installed and verify API key availability.

In [1]:
# Check KISS installation and version
import os
import sys
from IPython.display import display, Markdown

# Add src to path if running from the repo root
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from kiss import __version__

# Check available API keys
api_keys = {
    'GEMINI_API_KEY': os.environ.get('GEMINI_API_KEY'),
    'OPENAI_API_KEY': os.environ.get('OPENAI_API_KEY'),
    'ANTHROPIC_API_KEY': os.environ.get('ANTHROPIC_API_KEY'),
    'TOGETHER_API_KEY': os.environ.get('TOGETHER_API_KEY'),
    'OPENROUTER_API_KEY': os.environ.get('OPENROUTER_API_KEY'),
}

# Build status table
status_rows = "\n".join([
    f"| {key} | {'✓ Set' if value else '✗ Not set'} |"
    for key, value in api_keys.items()
])

display(Markdown(f"""
## KISS Framework Status

**Version:** `{__version__}`

### API Key Status

| Key | Status |
|-----|--------|
{status_rows}
"""))


## KISS Framework Status

**Version:** `0.1.14`

### API Key Status

| Key | Status |
|-----|--------|
| GEMINI_API_KEY | ✓ Set |
| OPENAI_API_KEY | ✓ Set |
| ANTHROPIC_API_KEY | ✓ Set |
| TOGETHER_API_KEY | ✓ Set |
| OPENROUTER_API_KEY | ✓ Set |


## 2. Your First Agent in 30 Seconds

Let's create a simple math agent that can evaluate expressions using tools.

KISS uses **native function calling** from the LLM providers. Your Python functions become tools automatically. Type hints become schemas. Docstrings become descriptions.

In [2]:
from IPython.display import display, Markdown
from kiss.core.kiss_agent import KISSAgent

def calculate(expression: str) -> str:
    """Evaluate a math expression."""
    return str(eval(expression))

agent = KISSAgent(name="Math Buddy")
result = agent.run(
    model_name="gemini-2.5-flash",
    prompt_template="Calculate: {question}",
    arguments={"question": "What is 15% of 847?"},
    tools=[calculate]
)
display(Markdown(f"**Result:** {result}"))

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Calculate: What is 15% of 847?                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Steps: 1/100, Tokens: 314/1048576, Budget: $0.0001/$10.00, Global Budget: $0.0001/$200.00                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── calculate ───────────────────────────────────────────────────╮
│ expression: 0.15 * 847                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────────────── OK ────────────────────────────────────────────────────────

127.05


...

╭──────────────────────────────────────────────────── Result ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  127.05                                                                                                         │
│                                                                                                                 │
╰────────────────────────────────────── steps=3  tokens=1561  cost=$0.0006 ───────────────────────────────────────╯

**Result:** 127.05

**Calculating the Expression**

I've formulated the core math problem as `0.15 * 847`. My next step is to use the calculation tool to obtain the 
numerical result, which will provide the answer to the user's question.

**Delivering the Result**

I've just finished the calculation, and the final answer I got is 127.05. I'm now preparing to communicate this 
outcome as the result of the inquiry.

15% of 847 is 127.05.

## 3. Custom Structured Output

Unlike other agentic systems, you do not need to specify the output schema for the agent. Just create a suitable "finish" function with parameters. The parameters could be treated as the top level keys in a json format.

In [3]:
import yaml
from IPython.display import display, Markdown
from kiss.core.kiss_agent import KISSAgent

def finish(success: bool, reasoning: str, output: str) -> str:
    """Finishes the current agent execution with success or failure, reasoning, and output.

    Args:
        success: True if the task was successful, False otherwise.
        reasoning: the reason you got the answer
        output: the final output as a number

    Returns:
        str: A YAML encoded dictionary containing 'success', 'reasoning', and 'output' keys.
    """
    result_str = yaml.dump(
        {
            "success": success,
            "reasoning": reasoning,
            "output": output,
        },
        indent=2,
        sort_keys=False,
    )
    return result_str
    
def calculate(expression: str) -> str:
    """Evaluate a math expression."""
    return str(eval(expression))

agent = KISSAgent(name="Math Buddy")
result = agent.run(
    model_name="gemini-2.5-flash",
    prompt_template="Calculate: {question}",
    arguments={"question": "What is 15% of 847?"},
    tools=[calculate, finish]
)
display(Markdown(f"""
### Structured Output Result

```yaml
{result}
```
"""))

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Calculate: What is 15% of 847?                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Steps: 1/100, Tokens: 355/1048576, Budget: $0.0002/$10.00, Global Budget: $0.0007/$200.00                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── calculate ───────────────────────────────────────────────────╮
│ expression: (15/100) * 847                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────────────── OK ────────────────────────────────────────────────────────

127.05


...

╭──────────────────────────────────────────────────── Result ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  "success": true                                                                                                │
│  "reasoning": |-                                                                                                │
│    I calculated 15% of 847 using the calculate tool.                                                            │
│  "output": |-                                                                                                   │
│    127.05                                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰────────────────────────────────────── steps=2  tokens=1129  cost=$0.0005 ───────────────────────────────────────╯


### Structured Output Result

```yaml
"success": true
"reasoning": |-
  I calculated 15% of 847 using the calculate tool.
"output": |-
  127.05

```


**Calculating the Percentage**

I've got the user's request: "15% of 847." I've formulated the calculation as (15/100) * 847. I'm preparing to use 
the `calculate` tool to evaluate this expression. I intend to present the answer back using the appropriate format 
once it is computed.

**Confirming the Calculation**

I have formalized the user's request, "15% of 847", into the equation (15/100) * 847. I'm ready to use the 
calculator to evaluate this expression. Once computed, I'll properly format and return the result.

**Calculating the Solution**

I've determined that 15% of 847 is 127.05. I'm now leveraging the `finish` tool to finalize and provide the 
solution.

## 4. Relentless Coding Agent

The Relentless Coding Agent is a single coding agent with auto-continuation that relentlessly works to complete complex coding tasks across multiple sub-sessions.

In [ ]:
from IPython.display import display, Markdown
from kiss.agents.coding_agents.relentless_coding_agent import RelentlessCodingAgent

# Create agent with a name
agent = RelentlessCodingAgent(name="My Coding Agent")

display(Markdown("""
## RelentlessCodingAgent Created Successfully!

**Key Features:**
- **Single-Agent Execution**: One coding agent with continuation support
- **Relentless Execution**: Auto-continues on failures with context
- **Path Access Control**: Enforces read/write permissions
- **Docker Support**: Optional container execution

---
### Running Example (requires API keys)
"""))

result = agent.run(
    prompt_template="Write, test, and optimize a fibonacci function",
    model_name="claude-sonnet-4-5",
    readable_paths=["src/"],
    writable_paths=["output/"],
    base_dir=".",
    max_steps=100,
    max_sub_sessions=3
)
display(Markdown(f"""
---
### Result

{result}
"""))



## RelentlessCodingAgent Created Successfully!

**Key Features:**
- **Single-Agent Execution**: One coding agent with continuation support
- **Relentless Execution**: Auto-continues on failures with context
- **Path Access Control**: Enforces read/write permissions
- **Docker Support**: Optional container execution

---
### Running Example (requires API keys)


Executing task: Write, test, and optimize a fibonacci function


╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  # Task                                                                                                         │
│  Write, test, and optimize a fibonacci function                                                                 │
│                                                                                                                 │
│  # CRITICAL Rules                                                                                               │
│  - Use Write() for new/full files. Edit() only for tiny fixes.                                                  │
│  - Bash(): set timeout_seconds=120 for test runs with sleeps/waits.                                             │
│  - Bash scripts with background jobs: use bounded poll loops (max iterations), never unbounded waits.           │
│  - IMMEDIATELY call finish(success=True, summary="done") once tests pass. NO extra verification.                │
│  - At step 98: finish(success=False, summary={"done":[...], "next":[...]})                                      │
│  - Work dir: /Users/ksen/Dropbox/work/kiss/artifacts/job_2026_02_13_12_45_15_148219/kiss_workdir                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Steps: 1/100, Tokens: 2526/200000, Budget: $0.0192/$200.00, Global Budget: $0.0202/$200.00                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Write ─────────────────────────────────────────────────────╮
│ /Users/ksen/Dropbox/work/kiss/artifacts/job_2026_02_13_12_45_15_148219/kiss_workdir/fibonacci.py                │
│    1 """                                                                                                        │
│    2 Optimized Fibonacci implementations with various approaches                                                │
│    3 """                                                                                                        │
│    4                                                                                                            │
│    5 def fibonacci_iterative(n):                                                                                │
│    6     """                                                                                                    │
│    7     Optimized iterative approach - O(n) time, O(1) space                                                   │
│    8     Best for most use cases                                                                                │
│    9     """                                                                                                    │
│   10     if n < 0:                                                                                              │
│   11         raise ValueError("n must be non-negative")                                                         │
│   12     if n <= 1:                                                                                             │
│   13         return n                                                                                           │
│   14                                                                                                            │
│   15     a, b = 0, 1                                                                                            │
│   16     for _ in range(2, n + 1):                                                                              │
│   17         a, b = b, a + b                                                                                    │
│   18     return b                                                                                               │
│   19                                                                                                            │
│   20                                                                                                            │
│   21 def fibonacci_memoized(n, memo=None):                                                                      │
│   22     """                                                                                                    │
│   23     Memoized recursive approach - O(n) time, O(n) space                                                    │
│   24     Good for recursive needs with caching                                                                  │
│   25     """                                                                                                    │
│   26     if memo is None:                                                                                       │
│   27         memo = {}                                                                                          │
│   28                                                                                                            │
│   29     if n < 0:                                                                                              │
│   30         raise ValueError("n must be non-negative")                                                         │
│   31     if n <= 1:                                                                                             │
│   32         return n                                                                                           │
│   33                                                  

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Steps: 19/100, Tokens: 138683/200000, Budget: $0.5177/$200.00, Global Budget: $0.5187/$200.00                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Bash ──────────────────────────────────────────────────────╮
│ Final verification test                                                                                         │
│ python3 /Users/ksen/Dropbox/work/kiss/artifacts/job_2026_02_13_12_45_15_148219/kiss_workdir/simple_test.py      │
│ timeout_seconds: 30                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────────────── OK ────────────────────────────────────────────────────────

Testing fibonacci implementations...

Testing iterative implementation:
  ✓ fib(0) = 0 (expected 0)
  ✓ fib(1) = 1 (expected 1)
  ✓ fib(2) = 1 (expected 1)
  ✓ fib(3) = 2 (expected 2)
  ✓ fib(4) = 3 (expected 3)
  ✓ fib(5) = 5 (expected 5)
  ✓ fib(6) = 8 (expected 8)
  ✓ fib(7) = 13 (expected 13)
  ✓ fib(8) = 21 (expected 21)
  ✓ fib(9) = 34 (expected 34)
  ✓ fib(10) = 55 (expected 55)

Testing memoized implementation:
  ✓ fib(0) = 0 (expected 0)
  ✓ fib(1) = 1 (expected 1)
  ✓ fib(2) = 1 (expected 1)
  ✓ fib(3) = 2 (expected 2)
  ✓ fib(4) = 3 (expected 3)
  ✓ fib(5) = 5 (expected 5)
  ✓ fib(6) = 8 (expected 8)
  ✓ fib(7) = 13 (expected 13)
  ✓ fib(8) = 21 (expected 21)
  ✓ fib(9) = 34 (expected 34)
  ✓ fib(10) = 55 (expected 55)

Testing matrix implementation:
  ✓ fib(0) = 0 (expected 0)
  ✓ fib(1) = 1 (expected 1)
  ✓ fib(2) = 1 (expected 1)
  ✓ fib(3) = 2 (expected 2)
  ✓ fib(4) = 3 (expected 3)
  ✓ fib(5) = 5 (expected 5)
  ✓ fib(6) = 8 (expected 8)
  ✓ fib(7) = 13 (expected 13)
  

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## 5. Versioning

The project uses semantic versioning (MAJOR.MINOR.PATCH). The version is defined in `src/kiss/_version.py`.

In [ ]:
from IPython.display import display, Markdown
from kiss import __version__

display(Markdown(f"**KISS Version:** `{__version__}`"))

### How to Run This Notebook

```bash
# From the kiss directory, run:
uv run jupyter notebook kiss.ipynb

# Or use JupyterLab:
uv run jupyter lab kiss.ipynb
```

### Running as a Script

You can also convert this notebook to a Python script and run it:

```bash
uv run jupyter nbconvert --to script kiss.ipynb
uv run python kiss.py
```